<a href="https://colab.research.google.com/github/DM871/dm871.github.io/blob/master/notebooks/lab_mip_1.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


## The Basics: Production Allocation

The following model is for a specific instance of the production 
allocation problem seen in the first lectures. We give here the primal
and its dual model with the instantiated numerical parameters.

$$\begin{array}{*{6}{r}l}
\max & 5x_1&+&6x_2&+&8x_3&=z\\
&6x_1&+&5x_2&+&10x_3 &\leq 60\\
&8x_1&+&4x_2&+&4x_3&\leq 40\\
&4x_1&+&5x_2&+&6x_3&\leq 50\\
&x_1,&&x_2,&&x_3&\geq 0
\end{array}$$

$$\begin{array}{*{6}{r}l}
\min & 60y_1&+&40y_2&+&50y_3&=u\\
&6y_1&+&8y_2&+&4y_3 &\leq 5\\
&5y_1&+&4y_2&+&5y_3&\leq 6\\
&10y_1&+&4y_2&+&6y_3&\leq 8\\
&y_1,&&y_2,&&y_3&\geq 0
\end{array}$$

## Analysis of the final tableau 

Solving one of the two problems provides the solution also to the other problem. The primal solution is
$x^*_1=0,x^*_2=7,x^*_3=2.5$ and the dual solution is
$y^*_1=0.2,y^*_2=0,y^*_3=1$. The objective value is $z^*=u^*=62$.

We can organize these values in a *tableau*. For the given problems it looks like this:

    |------+----+----+------+----+----+----+-----|
    |   x1 | x2 | x3 |   s1 | s2 | s3 | -z |   b |
    |------+----+----+------+----+----+----+-----|
    |   ?  |  1 |  0 |  ?   |  0 |  ? |  0 |   7 |
    |   ?  |  0 |  1 |  ?   |  0 |  ? |  0 | 5/2 |
    |   ?  |  0 |  0 |  ?   |  1 |  ? |  0 |   2 |
    |------+----+----+------+----+----+----+-----|
    | -0.2 |  0 |  0 | -0.2 |  0 | -1 |  1 | -62 |
    |------+----+----+------+----+----+----+-----|

A question mark is for the values that are not relevant for the goal of
this exercise.

The three numbers of the last row in the tableau above in the columns of
the variables that are not in basis are called *reduced costs*. They
indicate how much we should make each product more expensive in order to
be worth manufacturing it. The next three values are known as *shadow
prices*. After a change of sign they give the values of the dual
variables, which are interpreted as the *marginal values* of increasing
(or decreasing) the capacities of the resources (that is, the values by
which the objective function would improve if the corresponding constraints were
relaxed by one unit, which corresponds to buying one unit more of
that resource). In our example, which seeks maximization, the marginal value
1 for the third resource means that the objective function would
increase by 1 if we had one more unit of that resource.

It can be verified that in the primal problem at the optimum the first
and third resources are fully exhausted, that is, their constraint is
satisfied at the equality, while there is *slack* for the second
resource, that is, the constraint holds with strict inequality. Looking
at the marginal values, we see that the second resource has been given a
zero valuation. This seems plausible, since we are not using all the
capacity that we have, we are not willing to place much value on it
(buying one more unit of that resource would not translate in an
improvement of the objective function).

These results are captured by the Complementary Slackness theorem of
linear programming. If a constraint is not *"binding"* in the optimal
primal solution, the corresponding dual variable is zero in the optimal
solution to the dual model. Similarly, if a constraint in the dual model
is not *"binding"* in the optimal solution to the dual model, then the
corresponding variable is zero in the optimal solution to the primal
model.


## Solving the model with Python MPI

Let's write the primal model in Python and solve it with CBC.

In [57]:
%pip install mip

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Here is the script:

In [58]:
import mip

# Model
model = mip.Model("prod", solver_name=mip.CBC)

# Create decision variables
x1 = model.add_var(lb=0.0, ub=mip.INF, obj=5.0,
                  var_type=mip.CONTINUOUS, name="x1")  # arguments by name
x2 = model.add_var("x2", 0.0, mip.INF, 6.0, mip.CONTINUOUS)  # arguments by position
x3 = model.add_var(name="x3")  # arguments by deafult

# The objective is to maximize (this is redundant now, but it will overwrite Var declaration)
model.objective = mip.maximize(5.0*x1 + 6.0*x2 + 8.0*x3)

# Add the constraints to the model
model.add_constr(6.0*x1 + 5.0*x2 + 10.0*x3 <= 60.0, "c1")
model.add_constr(8.0*x1 + 4.0*x2 + 4.0*x3 <= 40.0, "c2")
model += 4.0*x1 + 5.0*x2 + 6.0*x3 <= 50.0, "c3"



The documentation for the functions `Model.add_var()` and
`Model.add_constr()`, as well as for all other functions in `python-mip` is
available from the [Online Manual](https://docs.python-mip.com/en/latest/index.html))
and more specifically from the [Class page](https://docs.python-mip.com/en/latest/classes.html)
For the variable $x_3$ the lower bound, upper bound, objective
coefficient and type are set to their default values that are,
respectively: `lb=0.0, ub=mip.INF, obj=0.0, var_type=mip.INF`.
Once the model has been built, it is good practice to inspect it to make sure that it has been built as we intended. To this goal see the description of the `model.write()` function below.

We are then ready to solve or optimize the model
(using `model.optimize`). You can then query the `x` attribute on the
variables to retrieve the solution (and the `name` attribute to
retrieve the variable name for each variable).

The variables of a model `model` can be queried as `model.vars` or `mip.VarList(model)`. The number of variables can be queried as `len(model.vars)` or as `model.num_cols`.
Specific variables can be retrieved by their indices or names. For example, to print the lower bounds of the first variable or of a varible named `z`, you can use, respectively: `model.vars[0].lb` or `model.vars['z'].lb`.



In [59]:
# Solve
model.optimize()

# Let's print the solution
for v in model.vars:
    print(v.name, v.x)

Starting solution of the Linear programming problem using Primal Simplex

Coin0506I Presolve 3 (0) rows, 3 (0) columns and 9 (0) elements
Clp1000I sum of infeasibilities 0 - average 0, 0 fixed columns
Coin0506I Presolve 3 (0) rows, 3 (0) columns and 9 (0) elements
Clp0006I 0  Obj 61.999976 Dual inf 2020 (3)
Clp0029I End of values pass after 3 iterations
Clp0000I Optimal - objective value 62
Clp0000I Optimal - objective value 62
Clp0000I Optimal - objective value 62
Clp0032I Optimal objective 62 - 0 iterations time 0.002
x1 0.0
x2 6.999999999999998
x3 2.500000000000001


In the definition of the model `model = mip.Model("prod", solver_name=mip.CBC)` we have specified that we want to model to be solved with CBC. If we have also gurobi installed the default will be for gurobi. We can ascertain which solver is actually used by reading the output produced by the function `optimize`. Here it is CBC.

The screen output of CBC keeps you informed of the successes (or failures) of the attempts the solver makes 
continually prints messages containing details about the current search status.
Understanding these messages is a key step to pinpoint which are the main difficulties in solving your problem. Once you have this information in your hands you can start to tune CBC so that it will perform better considering the type of problem you are working on.
You find some information on this regard in the documentation about CBC, for example [here](https://raw.githubusercontent.com/coin-or/COIN-OR-OptimizationSuite/master/Installer/files/doc/cbcCommandLine.pdf).

In the output above we observe that initially two rounds of presolving are applied (`Coin0506I`) then the LP solver is started (Clp) and an optimal solution is found. In integer programs lines starting by `Cbc` indicate that solution techinques for integer programming (cuts, branch and bound, heuristics) are applied.   
The optimal solution is found with value 62.

Most of the information associated with a python-mip model is stored in a
set of attributes. Some attributes are associated with the variables of
the model, some with the constraints of the model, and some with the
model itself. To give a simple example, solving an optimization model
causes the `x` variable attribute to be populated.

Attributes can be accessed in two ways in the Python interface. The
first is to use the `getattr(object, name)` and `setattr(object, name, value)` methods, which are
available on variables (`getattr(Var, ... )`/`setattr(Var, ...)`), linear constraints
(`getattr(Constr, ...)`/`setattr(Constr, ...)`), and models
(`getattr(Model, ...)`/`setattr(Model, ...)`). These are called with the attribute
name as the first argument (e.g., `getattr(var, "x")` or
`setAttr(constr, "rhs", 0.0))`. Attributes can also be accessed more
directly: you can follow an object name by a period, followed by the
name of an attribute of that object. Note that upper/lower case is
ignored when referring to attributes. Thus, `b = constr.rhs` is
equivalent to `b = getattr(constr, "rhs")`, and `constr.rhs = 0.0` is
equivalent to `setattr(constr, "rhs", 0.0)`. The full list of available
attributes can be found following the [Class page](https://docs.python-mip.com/en/latest/classes.html).

Note that the code above can also be put in a script file called `prod1.py`. Then we can solve the model
by calling:

```
    > python prod1.py
```
If something goes wrong check that `mip` is available at the
import, that the file is saved in the correct place and that there are no syntax errors.

#### The Value of the Dual variables

The value of the dual variables can be accessed by referring to the
attribute `pi` of the corresponding constraints. The list of constraints of a model `model` can be accessed via `model.constrs` or `mip.ConstrList(model)`.

In [60]:
# Let's print the dual variables and the slack
for c in model.constrs:
    print(c.name, c.pi, c.slack)

c1 0.19999999999999965 0.0
c2 -0.0 2.0
c3 1.0000000000000004 7.105427357601002e-15


The values `c1`,`c2`,`c3` are the shadow prices (here 0.2, 0.0 and 1) which correspond to
the marginal value of the resources. The c1 and c3 constraints' value is
different from zero. This indicates that there's a variable on the upper
bound for those constraints, or in other terms that these constraints
are *"binding"*. The second constraint is not *"binding"*.

#### Your Task

Try relaxing the value of each binding constraint one at a time, solve
the modified problem, and see what happens to the optimal value of the
objective function. Also check that, as expected, changing the value of
non-binding constraints won't make any difference to the solution.

#### Your Task

We can also access several quantities associated with the variables. A
particularly relevant one is the *reduced cost*. Print the reduced costs
of the variables for our example and make sure that they correspond to
the expected values from the tableau above. \[Hint: look for the
variable attribute `rc`.\] What can we say about the solution found on
the basis of the reduced costs?

Let's now focus on the values output during the execution of the
simplex. Let's first solve another small numerical example:


In [61]:
import mip

m = mip.Model("infeas", sense=mip.MAXIMIZE, solver_name=mip.CBC)

x = m.add_var(name="x") # ie, >= 0 (default lb=0, ub=mip.INF)
y = m.add_var(name="y") # ie, >= 0

m.objective = x - y

m += x + y <= 2, "c1"
m += 2*x + 2*y >= 5, "c2"

status = m.optimize()
print(status)

Starting solution of the Linear programming problem using Primal Simplex

Coin0506I Presolve 0 (-2) rows, 0 (-2) columns and 0 (-4) elements
Clp0000I Optimal - objective value 2
Coin0511I After Postsolve, objective 2, infeasibilities - dual 0 (0), primal 0.999999 (1)
Coin0512I Presolved model was optimal, full model needs cleaning up
Clp0006I 0  Obj 2 Primal inf 0.999999 (1)
Clp0001I Primal infeasible - objective value 2
Clp1000I Dual after postsolve not optimal! - trying primal
Clp0001I Primal infeasible - objective value 2
Clp0032I PrimalInfeasible objective 2 - 0 iterations time 0.002, Presolve 0.00, Idiot 0.00
OptimizationStatus.INFEASIBLE



This means that the presolve process has identified the model as infeasible or unbounded. To discover which of
the two CBC runs the simplex.

See the documetation of [OptimizationStatus](https://docs.python-mip.com/en/latest/classes.html#optimizationstatus) for the possible statuses.


## Parameters

The optimization process in Python-mip can be controlled by setting a number of parameters (some of them have sense only in integer problems):
- the solver specified via `solver_name` in `mip.Model`
- search limits specified via the function `optimize(max_seconds=inf, max_nodes=1073741824, max_solutions=1073741824, max_seconds_same_incumbent=inf, max_nodes_same_incumbent=1073741824, relax=False)`
- whether to `preprocess` or not
- the [emphasis of the search](https://docs.python-mip.com/en/latest/classes.html#mip.Model.emphasis)
- the [LP_Method](https://docs.python-mip.com/en/latest/classes.html#lp-method)

In [62]:
m.preprocess=0 #  -1 means automatic, 0 means off and 1 means on.
m.optimize()

if m.status == mip.OptimizationStatus.INFEASIBLE:
    # Turn presolve off to determine whether m is infeasible or unbounded
    m.preprocess=0 #  -1 means automatic, 0 means off and 1 means on
    m.LP_Method=2 # 0=automatic, 1=dual simplex, 2=primal simplex, 3=barrier    
    m.optimize()
elif m.status == mip.OptimizationStatus.OPTIMAL:
    print('Optimal objective: %g' % m.objVal)
    print( m.vars["x"].x )
    print( mip.xi(m.vars["y"]) )
    exit(0)

Starting solution of the Linear programming problem using Primal Simplex

Clp0006I 0  Obj 2 Primal inf 0.999999 (1)
Clp0001I Primal infeasible - objective value 2
Starting solution of the Linear programming problem using Primal Simplex

Clp0006I 0  Obj 2 Primal inf 0.999999 (1)
Clp0001I Primal infeasible - objective value 2


Resolving gives us the needed information. The `Primal infeas.` is the
objective function of the auxiliary problem in the phase I of the
two-phase simplex method. The Primal Infeas. never reaches 0. This means
that a feasible solution cannot be found and the problem is therefore
infeasible. Try to change 5 to 2 in the right-hand-side of the second
constraint of the model above. What happens? Explain the behaviour.



#### Your Task

Resolve the first production allocation example above using the primal method.
Compare the iteration values of the simplex with the previous ones. Give
your interpretation of the values Objective, Primal Inf. and Dual Inf.

Python-mip makes also available a function for collecting several features of the problem to solve:

In [63]:
dict(zip(mip.features(), mip.compute_features(m)))


{'cols': 2.0,
 'rows': 2.0,
 'colsPerRow': 1.0,
 'equalities': 0.0,
 'nzEqualities': 0.0,
 'percEqualities': 0.0,
 'percNzEqualities': 0.0,
 'inequalities': 2.0,
 'nzInequalities': 4.0,
 'nz': 4.0,
 'density': 100.0,
 'bin': 0.0,
 'genInt': 0.0,
 'integer': 0.0,
 'continuous': 2.0,
 'percInteger': 0.0,
 'percBin': 0.0,
 'nUnbounded1': 2.0,
 'percUnbounded1': 100.0,
 'nUnbounded2': 0.0,
 'percUnbounded2': 0.0,
 'rPartitioning': 0.0,
 'rPercPartitioning': 0.0,
 'rPacking': 0.0,
 'rPercPacking': 0.0,
 'rPartPacking': 0.0,
 'rPercRowsPartPacking': 0.0,
 'rCovering': 0.0,
 'rPercCovering': 0.0,
 'rCardinality': 0.0,
 'rPercCardinality': 0.0,
 'rKnapsack': 0.0,
 'rPercKnapsack': 0.0,
 'rIntegerKnapsack': 0.0,
 'rPercIntegerKnapsack': 0.0,
 'rInvKnapsack': 0.0,
 'rPercInvKnapsack': 0.0,
 'rSingleton': 0.0,
 'rPercSingleton': 0.0,
 'rAggre': 0.0,
 'rPercAggre': 0.0,
 'rPrec': 0.0,
 'rPercPrec': 0.0,
 'rVarBnd': 0.0,
 'rPercVarBnd': 0.0,
 'rBinPacking': 0.0,
 'rPercBinPacking': 0.0,
 'rMixedBin

#### Exporting Model Data to a File

The function `model.write(filename)` writes model data to a file. The file type
is encoded in the extension of the file name passed to the function. Valid
extensions for writing the model itself are:

- `.lp` mip model stored in the [LP file format](https://www.ibm.com/docs/en/icos/12.9.0?topic=problem-using-lp-format)
- `.mps` mip model stored in the [MPS file format](https://en.wikipedia.org/wiki/MPS_(format))
- `.sol` initial integer feasible solution
- `.bas` [optimal basis](https://lpsolve.sourceforge.net/5.5/bas-format.htm) for the linear programming relaxation.



#### Your Task

Try all these formats on the production allocation example above and
check their contents. The MPS file is not very user friendly. This is
because the format is an old format when computer technology had much
more limitations than nowadays. The CPLEX-LP format is a more explicit
version of the problem that may be useful to check whether the model we
implemented in Python is actually the one we intended.

If you have any of them installed, try solving the problem with other
solvers, eg, `cplex`, `glpk` and `scip`. For this you have to use the
MPS (Mathematical Programming System) format and run the following: 
```
    cplex -c read prod.mps optimize
    glpsol --mps prod.mps 
    scip -f prod.mps 
```
Gurobi has also a [command-line tool](https://www.gurobi.com/documentation/9.1/refman/grb_command_line_tool.html)
to solve model files:
```
    gurobi_cl model.mps
```
You may also use the online solver at NEOS, the Network Enabled
Optimization Server supported by the US federal government and located
at Argonne National Lab. To submit an MPS model to NEOS visit
<http://www.neos-server.org/neos/>, click on the icon "NEOS Solvers",
scroll down to the Linear Programming or Mixed Integer Linear
Programming list, click on one of those, scroll down to "Model File",
click on "Choose File", select a file from your computer that contains
an MPS model, scroll down to "e-mail address:", type in your email
address, and click Submit to NEOS.

[Standard MPS files do not indicate whether to minimize or maximize the objective. Thus your MPS files will come out the same whether the objective is minimize or maximize.

As you are seeing, most solvers minimize the objective by default. A solver may have a switch to tell it to maximize instead, but that is different for each solver.

If you change the signs of all the objective coefficients, while leaving the constraints unchanged, then minimizing the resulting MPS file will be equivalent to maximizing the original problem. This can be done easily by putting the entire objective expression in parentheses and placing a minus sign in front of it.]